In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'project-mimic-430923'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [15]:
# 1. Get basic patient info (admissions + demographics + death status)
patient_info_query = """
SELECT adm.subject_id, adm.hadm_id, adm.admittime, adm.dischtime, pat.gender, pat.anchor_age AS age, adm.race, adm.hospital_expire_flag
FROM `physionet-data.mimiciv_hosp.admissions` AS adm
JOIN `physionet-data.mimiciv_hosp.patients` AS pat
ON adm.subject_id = pat.subject_id
WHERE adm.admittime IS NOT NULL
"""
patient_info_df = run_query(patient_info_query)

patient_info_df['in_hospital_death'] = patient_info_df['hospital_expire_flag'] == 1

patient_info_df = patient_info_df.drop(columns=['hospital_expire_flag'])

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [16]:
patient_info_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death
0,14962874,22614141,2165-10-15 15:59:00,2165-10-18 16:47:00,M,89,WHITE,False
1,13241600,22574379,2162-10-29 15:36:00,2162-10-30 16:00:00,M,89,WHITE,False
2,12945423,20185299,2122-07-08 05:40:00,2122-07-19 15:15:00,M,89,WHITE,False
3,16014534,21633959,2176-03-03 14:33:00,2176-03-11 13:57:00,F,89,WHITE,False
4,16014534,27188276,2176-03-27 11:53:00,2176-03-31 14:10:00,F,89,WHITE,False
...,...,...,...,...,...,...,...,...
431226,12545126,27135177,2187-11-01 15:47:00,2187-11-02 09:45:00,M,91,WHITE,False
431227,17579295,26848807,2162-11-16 07:15:00,2162-11-19 14:27:00,M,91,WHITE,False
431228,18563244,26485584,2136-08-18 14:48:00,2136-08-19 11:00:00,F,91,WHITE,False
431229,14865704,29199399,2156-11-12 13:59:00,2156-11-13 14:00:00,F,91,OTHER,False


In [17]:
# Count number of patients
num_patients = patient_info_df['subject_id'].nunique()
print(f"Number of patients: {num_patients}")

Number of patients: 180733


In [18]:
# 2. Get item IDs for lipase tests in icu
lipase_item_query = """
SELECT itemid, label, category
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df = run_query(lipase_item_query)
print(lipase_items_df)
lipase_itemids = "225672"  # Lipase item IDs

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   itemid   label category
0  225672  Lipase     Labs


In [19]:
# 3. Retrieve lipase values
lipase_values_query_icu = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN ({lipase_itemids})
ORDER BY subject_id, charttime
"""
lipase_values_df_icu = run_query(lipase_values_query_icu)

# Merge lipase values with patient info and clean data
merged_df = pd.merge(lipase_values_df_icu, patient_info_df, on=['subject_id', 'hadm_id'], how='left')
merged_df.dropna(subset=['age', 'lipase_level'], inplace=True)

# Identify patients with high lipase levels
def check_lipase(row):
    upper_limit = 140 if row['age'] < 60 else 151
    return row['lipase_level'] >= 3 * upper_limit

high_lipase_df_icu = merged_df[merged_df.apply(check_lipase, axis=1)]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [20]:
high_lipase_df_icu

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death
1,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False
15,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
16,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
55,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False
56,10036086,28728587,2196-05-28 03:39:00,777.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False
...,...,...,...,...,...,...,...,...,...,...
17140,19882958,29628147,2182-08-31 03:10:00,787.0,2182-08-29 17:25:00,2182-09-03 13:50:00,M,83,WHITE,False
17167,19899716,21665899,2143-09-03 17:16:00,587.0,2143-09-03 12:59:00,2143-09-08 18:00:00,M,62,BLACK/AFRICAN AMERICAN,False
17178,19901341,23906609,2169-08-10 06:48:00,588.0,2169-08-06 14:52:00,2169-09-05 14:58:00,F,55,WHITE,False
17189,19907884,24707264,2181-01-24 04:42:00,789.0,2181-01-23 21:32:00,2181-02-03 13:30:00,F,38,WHITE,False


In [21]:
# Count number of patients with high lipase levels
num_patients = high_lipase_df_icu['subject_id'].nunique()
print(f"Number of unique patients with high lipase levels: {num_patients}")

Number of unique patients with high lipase levels: 594


In [22]:
patient_info_df['admittime'] = pd.to_datetime(patient_info_df['admittime'])
patient_info_df['dischtime'] = pd.to_datetime(patient_info_df['dischtime'])
patient_info_df['length_of_stay'] = (patient_info_df['dischtime'] - patient_info_df['admittime']).dt.total_seconds() / (60 * 60 * 24)

In [23]:
high_lp_icu_df = pd.merge(high_lipase_df_icu, patient_info_df[['subject_id', 'hadm_id', 'length_of_stay']], on = ['subject_id', 'hadm_id'], how='left')

In [24]:
def select_row(group):
    # Check if there's any row with hospital_expire_flag = 1
    if (group['in_hospital_death']).any():
        # Return the first row with hospital_expire_flag = 1
        return group[group['in_hospital_death']].iloc[0]
    else:
        # Otherwise, return the row with the maximum length_of_stay
        return group.loc[group['length_of_stay'].idxmax()]

# Apply the function to each group and reset the index
high_lp_icu_filtered_df = high_lp_icu_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-24-f4456c3b9736>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_lp_icu_filtered_df = high_lp_icu_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [25]:
high_lp_icu_filtered_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556
1,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944
2,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528
3,10049833,20762302,2168-05-24 17:37:00,692.0,2168-05-24 16:43:00,2168-05-30 12:05:00,M,32,WHITE,False,5.806944
4,10057482,25416257,2145-04-04 15:26:00,1429.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
...,...,...,...,...,...,...,...,...,...,...,...
589,19882958,29628147,2182-08-31 03:10:00,787.0,2182-08-29 17:25:00,2182-09-03 13:50:00,M,83,WHITE,False,4.850694
590,19899716,21665899,2143-09-03 17:16:00,587.0,2143-09-03 12:59:00,2143-09-08 18:00:00,M,62,BLACK/AFRICAN AMERICAN,False,5.209028
591,19901341,23906609,2169-08-10 06:48:00,588.0,2169-08-06 14:52:00,2169-09-05 14:58:00,F,55,WHITE,False,30.004167
592,19907884,24707264,2181-01-24 04:42:00,789.0,2181-01-23 21:32:00,2181-02-03 13:30:00,F,38,WHITE,False,10.665278


In [26]:
high_lp_icu_filtered_df.to_csv('High_Lipase_ICU_Dataset.csv', encoding='utf-8', index = False)

In [35]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in high_lp_icu_filtered_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(high_lp_icu_filtered_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
2    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
3    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
4    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   

            dischtime gender  age   race  in_hospital_death  length_of_stay  \
0 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
1 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
2 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
3 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
4 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   

  icd_code  seq_num  icd_version  
0  

In [36]:
merged_comorbidities_df.to_csv('High_Lipase_ICU_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [27]:
ct_item_query = """
SELECT itemid, label
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%computed tomography%'
   OR LOWER(label) LIKE '%abdomen%'
   OR LOWER(label) LIKE '%pelvis%'
   OR LOWER(label) LIKE '%scan%'
"""
ct_items_df = run_query(ct_item_query)
print(ct_items_df)
ct_itemids = [221214, 229582]  # CT item IDs

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   itemid                                      label
0  225999               Vanilla Scandi Shake (mixed)
1  226000             Chocolate Scandi Shake (mixed)
2  226002  Vanilla Lactose Free Scandi Shake (mixed)
3  221214                                    CT scan
4  225461                                     Pelvis
5  229582                           Portable CT scan
6  228714                            Bladder scanned
7  229371                      Bladder Scan Estimate
8  220462                                   Scandium


In [28]:
ct_scan_query = f"""
SELECT subject_id, hadm_id, stay_id, starttime, endtime, itemid, value
FROM `physionet-data.mimiciv_icu.procedureevents`
WHERE itemid IN ({', '.join(map(str, ct_itemids))})
ORDER BY subject_id, starttime
"""
ct_scan_df = run_query(ct_scan_query)

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [29]:
ct_scan_df

,subject_id,hadm_id,stay_id,starttime,endtime,itemid,value
0,10001217,24597018,37067082,2157-11-21 14:00:00,2157-11-21 14:01:00,221214,1.0
1,10001217,27703517,34592300,2157-12-19 20:19:00,2157-12-19 20:20:00,221214,1.0
2,10001884,26184834,37510196,2131-01-11 15:15:00,2131-01-11 15:16:00,221214,1.0
3,10002155,28994087,31090461,2130-09-26 12:45:00,2130-09-26 12:46:00,221214,1.0
4,10002428,28662225,38875437,2156-04-19 23:00:00,2156-04-19 23:01:00,221214,1.0
...,...,...,...,...,...,...,...
22038,19999068,21606769,30143796,2161-08-27 17:35:00,2161-08-27 17:36:00,221214,1.0
22039,19999442,26785317,32336619,2148-11-19 19:45:00,2148-11-19 19:46:00,221214,1.0
22040,19999442,26785317,32336619,2148-11-20 05:26:00,2148-11-20 05:27:00,221214,1.0
22041,19999828,25744818,36075953,2149-01-09 21:30:00,2149-01-09 21:31:00,221214,1.0


In [30]:
print(ct_scan_df['subject_id'].nunique())

12494


In [38]:
merged_ct_icd_df = pd.merge(high_lp_icu_df, ct_scan_df, on=['subject_id', 'hadm_id'], how='inner')

# Merge length of stay into the dataset
merged_ct_icd_df = pd.merge(merged_ct_icd_df, patient_info_df[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='left')

In [39]:
merged_ct_icd_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,stay_id,starttime,endtime,itemid,value
0,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,35526828,2159-09-28 16:36:00,2159-09-28 16:37:00,221214,1.0
1,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,35526828,2159-10-05 18:20:00,2159-10-05 18:21:00,221214,1.0
2,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,35526828,2159-10-12 17:00:00,2159-10-12 17:01:00,221214,1.0
3,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,35526828,2159-09-28 16:36:00,2159-09-28 16:37:00,221214,1.0
4,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,35526828,2159-10-05 18:20:00,2159-10-05 18:21:00,221214,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,19655310,23438001,2147-03-18 14:03:00,669.0,2147-03-14 07:15:00,2147-05-12 17:18:00,F,41,WHITE,False,59.418750,31620848,2147-04-04 16:00:00,2147-04-04 16:01:00,221214,1.0
1759,19669999,28614555,2148-08-10 06:28:00,1208.0,2148-08-10 02:24:00,2148-08-10 13:55:00,F,84,OTHER,True,0.479861,37577874,2148-08-10 09:00:00,2148-08-10 09:01:00,221214,1.0
1760,19676211,27134485,2134-05-15 05:26:00,980.0,2134-04-22 20:37:00,2134-05-21 18:57:00,F,79,WHITE,True,28.930556,38939076,2134-05-15 17:12:00,2134-05-15 17:13:00,221214,1.0
1761,19720119,28060710,2180-12-07 14:56:00,757.0,2180-12-01 00:04:00,2180-12-20 13:30:00,F,45,WHITE,False,19.559722,35883202,2180-12-01 21:15:00,2180-12-01 21:16:00,221214,1.0


In [40]:
high_lp_icu_ct_df = merged_ct_icd_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-40-22127e64232f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_lp_icu_ct_df = merged_ct_icd_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [42]:
high_lp_icu_ct_df.to_csv('High_Lipase_ICU_CT_Dataset.csv', encoding='utf-8', index = False)

In [43]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in high_lp_icu_ct_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(high_lp_icu_ct_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
1    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
4    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   

            dischtime gender  age   race  in_hospital_death  length_of_stay  \
0 2159-10-24 13:40:00      M   63  WHITE              False       31.756944   
1 2159-10-24 13:40:00      M   63  WHITE              False       31.756944   
2 2159-10-24 13:40:00      M   63  WHITE              False       31.756944   
3 2159-10-24 13:40:00      M   63  WHITE              False       31.756944   
4 2159-10-24 13:40:00      M   63  WHITE              False       31.756944   

    stay_id           starttime       

In [44]:
merged_comorbidities_df.to_csv('High_Lipase_ICU_CT_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [45]:
ap_icd_query = f"""
SELECT subject_id, hadm_id, icd_code
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'K85%'
ORDER BY subject_id
"""
ap_icd_df = run_query(ap_icd_query)

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [46]:
ap_icd_df

,subject_id,hadm_id,icd_code
0,10002807,28464737,K8590
1,10004606,29242151,K8510
2,10006431,24638489,K8580
3,10008816,22267961,K859
4,10012206,23961896,K8510
...,...,...,...
1872,19939903,26616095,K851
1873,19968351,23732375,K8590
1874,19972266,24170910,K8590
1875,19990545,23106222,K8590


In [47]:
ap_icd_df['subject_id'].nunique()

1401

In [48]:
merged_ct_ap_icd_df = pd.merge(merged_ct_icd_df, ap_icd_df, on=['subject_id', 'hadm_id'], how='inner')

# Merge length of stay into the dataset
merged_ct_ap_icd_df = pd.merge(merged_ct_ap_icd_df, patient_info_df[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='left')

In [49]:
merged_ct_ap_icd_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,stay_id,starttime,endtime,itemid,value,icd_code
0,10314359,29423991,2127-09-06 01:47:00,575.0,2127-09-04 05:55:00,2127-10-16 16:27:00,M,27,WHITE,False,42.438889,30481610,2127-09-13 13:45:00,2127-09-13 13:46:00,221214,1.0,K859
1,10314359,29423991,2127-09-06 01:47:00,575.0,2127-09-04 05:55:00,2127-10-16 16:27:00,M,27,WHITE,False,42.438889,30481610,2127-10-03 12:58:00,2127-10-03 12:59:00,221214,1.0,K859
2,10354450,28518599,2163-08-28 22:10:00,1113.0,2163-08-28 21:01:00,2163-11-01 18:28:00,M,30,OTHER,False,64.893750,39932818,2163-09-15 17:21:00,2163-09-15 17:22:00,221214,1.0,K852
3,10354450,28518599,2163-08-28 22:10:00,1113.0,2163-08-28 21:01:00,2163-11-01 18:28:00,M,30,OTHER,False,64.893750,39932818,2163-09-19 16:10:00,2163-09-19 16:11:00,221214,1.0,K852
4,10354450,28518599,2163-08-28 22:10:00,1113.0,2163-08-28 21:01:00,2163-11-01 18:28:00,M,30,OTHER,False,64.893750,39932818,2163-09-24 09:00:00,2163-09-24 09:01:00,221214,1.0,K852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,19655310,23438001,2147-03-18 14:03:00,669.0,2147-03-14 07:15:00,2147-05-12 17:18:00,F,41,WHITE,False,59.418750,31620848,2147-03-29 17:00:00,2147-03-29 17:01:00,221214,1.0,K8512
208,19655310,23438001,2147-03-18 14:03:00,669.0,2147-03-14 07:15:00,2147-05-12 17:18:00,F,41,WHITE,False,59.418750,31620848,2147-04-04 16:00:00,2147-04-04 16:01:00,221214,1.0,K8512
209,19676211,27134485,2134-05-15 05:26:00,980.0,2134-04-22 20:37:00,2134-05-21 18:57:00,F,79,WHITE,True,28.930556,38939076,2134-05-15 17:12:00,2134-05-15 17:13:00,221214,1.0,K8590
210,19720119,28060710,2180-12-07 14:56:00,757.0,2180-12-01 00:04:00,2180-12-20 13:30:00,F,45,WHITE,False,19.559722,35883202,2180-12-01 21:15:00,2180-12-01 21:16:00,221214,1.0,K8590


In [50]:
# Count the number of patients with high lipase levels who had a CT scan
num_patients_ap_icd_ct = merged_ct_ap_icd_df['subject_id'].nunique()
print(f"Number of patients with high lipase levels who had a CT scan: {num_patients_ap_icd_ct}")

# Count the number of rows in the dataset
num_rows = merged_ct_ap_icd_df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of patients with high lipase levels who had a CT scan: 40
Number of rows in the dataset: 212


In [51]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in merged_ct_ap_icd_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(merged_ct_ap_icd_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10314359  29423991 2127-09-06 01:47:00         575.0 2127-09-04 05:55:00   
1    10314359  29423991 2127-09-06 01:47:00         575.0 2127-09-04 05:55:00   
2    10314359  29423991 2127-09-06 01:47:00         575.0 2127-09-04 05:55:00   
3    10314359  29423991 2127-09-06 01:47:00         575.0 2127-09-04 05:55:00   
4    10314359  29423991 2127-09-06 01:47:00         575.0 2127-09-04 05:55:00   

            dischtime gender  age   race  in_hospital_death  length_of_stay  \
0 2127-10-16 16:27:00      M   27  WHITE              False       42.438889   
1 2127-10-16 16:27:00      M   27  WHITE              False       42.438889   
2 2127-10-16 16:27:00      M   27  WHITE              False       42.438889   
3 2127-10-16 16:27:00      M   27  WHITE              False       42.438889   
4 2127-10-16 16:27:00      M   27  WHITE              False       42.438889   

    stay_id           starttime       

In [52]:
# Count the number of unique patients who died in the hospital
num_patients_with_death_info = merged_comorbidities_df[merged_comorbidities_df['in_hospital_death'] == True][['subject_id', 'hadm_id']].drop_duplicates().shape[0]
print(f"Number of unique patients who died in the hospital: {num_patients_with_death_info}")

Number of unique patients who died in the hospital: 12
